# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [96]:
import sys

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [97]:
output = open("./some_test_data.txt", 'w')

In [98]:
output.write('1234555545')

10

In [99]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [100]:
from datetime import datetime

original_write = sys.stdout.write

def my_write(string_text):
    new_text =  '[' + str(datetime.now()) + ']: ' + string_text 
    original_write(new_text)
    return len(new_text.encode('utf-8'))

sys.stdout.write = my_write

In [101]:
print('1, 2, 3')

[2023-01-10 14:32:03.791220]: 1, 2, 3[2023-01-10 14:32:03.792218]: 


In [102]:
sys.stdout.write('1, 2, 3')

[2023-01-10 14:32:04.982863]: 1, 2, 3

37

In [103]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [112]:
def timed_output(function):
    def wrapper(*args, **kwargs):
        new_string = '[' + str(datetime.now()) + ']: ' + function(*args, **kwargs)
        print(new_string)
        return len(new_string.encode('utf-8'))
    return wrapper

In [113]:
@timed_output
def print_greeting(name):
    return f'Hello, {name}!'

In [114]:
print_greeting("Nikita")

[2023-01-10 14:32:48.236414]: Hello, Nikita!


44

Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [27]:
def redirect_output(filepath):
    def decorator(func):
        def wrapper(*args):
            with open(filepath, 'w') as f:
                original_stdout = sys.stdout
                sys.stdout = f
                func(*args)
            sys.stdout = original_stdout
        return wrapper
    return decorator

In [116]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [117]:
calculate()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 


In [30]:
%cat function_output.txt

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 
